<a href="https://colab.research.google.com/github/Simaregele/TrainProject/blob/master/CianHH_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [0]:
df = pd.DataFrame() 
main_df = pd.DataFrame() 

In [2]:
# генерим странички по квартирам: студия или однушка, аренда на длительный срок москва
# это список ЮРЛ страниц с карчтоками
url_list = []
for i in range(1, 54):
  url = "https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=flat&p={}&region=1&room1=1&room9=1&type=4".format(str(i))
  print(url)
  url_list.append(url)

https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=flat&p=1&region=1&room1=1&room9=1&type=4
https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=flat&p=2&region=1&room1=1&room9=1&type=4
https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=flat&p=3&region=1&room1=1&room9=1&type=4
https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=flat&p=4&region=1&room1=1&room9=1&type=4
https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=flat&p=5&region=1&room1=1&room9=1&type=4
https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=flat&p=6&region=1&room1=1&room9=1&type=4
https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=flat&p=7&region=1&room1=1&room9=1&type=4
https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=flat&p=8&region=1&room1=1&room9=1&type=4
https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=flat&p=9&region=1&room1=1&room9=1

In [0]:
new_page = requests.get(url_list[0])

In [4]:
new_page

<Response [200]>

In [0]:
soup = BeautifulSoup(new_page.text, 'html.parser')

In [0]:
card = soup.find_all(class_='_93444fe79c--card--_yguQ')[0]
card_title = soup.find_all('a', class_="c6e8ba5398--header--1fV2A")

In [205]:
# Берем ЮРЛ карточки
def get_cards_url(card):
  card_url = card.find('a', class_="c6e8ba5398--header--1fV2A", href=True)['href']
  return card_url
get_cards_url(card)

'https://www.cian.ru/rent/flat/224019372/'

In [0]:
def get_page_from_card(card_url):
  # переходим на страниwу и парсим все данные со страницы
  get_page = requests.get(card_url)
  card_page = BeautifulSoup(get_page.text, 'html.parser')
  return card_page

In [0]:
#извлек название и адрес
title = card_page.find('h1', class_="a10a3f92e9--title--2Widg").get_text()
adress = card_page.find('address', class_= "a10a3f92e9--address--140Ec").get_text()
df['title'] = [title]
df['adress'] = [adress] 

In [0]:
# станции метро
metro_station = card_page.find_all('li', class_="a10a3f92e9--underground--kONgx")
m_stations = []
for i in metro_station:
  m_stations.append(i.get_text())

df['metro'] = [m_stations]


In [200]:
m_stations

['Бауманская ⋅  5 мин. пешком',
 'Красносельская ⋅  10 мин. пешком',
 'Комсомольская ⋅  19 мин. пешком']

In [0]:
# остальная инфа 
all_apartments_info = card_page.find_all('div', class_="a10a3f92e9--info-text--2uhvD")
apartments_area = apartments_info[0].get_text()
living_apartmetn_area = apartments_info[1].get_text()
kitche_area = apartments_info[2].get_text()
apartments_level = apartments_info[3].get_text()
year_of_construction = apartments_info[4].get_text()
apartsments_text_subscription = card_page.find('p', class_="a10a3f92e9--description-text--1_Lup").get_text()
df['apartments_area'] = [apartments_area]
df['living_apartmetn_area'] = [living_apartmetn_area]
df['kitche_area'] = [kitche_area]
df['apartments_level'] = [apartments_level]
df['year_of_construction'] = [year_of_construction]
df['apartsments_text_subscription'] = [apartsments_text_subscription]

In [39]:
apartsments_text_subscription

'Покажу оперативно! ЖК "Трибека", корпус Fulton Дом бизнес-класса с своей территорией и круглосуточной охраной. Вся инфраструтура- магазины, рестораны, химчистка, салоны красоты, фитнес- центр на территории комплекса. Апартамент студийного плана - гостиная совмещена с кухней, гардеробная комната, санузел совмещенный. Мебель, сантехника, бытовая техника премиум класса. Домашний кинотеатр с супер звуком. Полы плитка, натуральный дуб, встроенные в пол отопительные конвекторы с возможностью регулирования температуры, кондиционер. Установлен высокоскоростной интернет роутер. Есть все необходимое для жизни, включая посуду и постельное белье'

In [0]:
additional_appartments_info = card_page.find('ul', class_="a10a3f92e9--container--L-EIV")
appartments_features = []
for i in additional_appartments_info:
  appartments_features.append(i.get_text())

df['appartments_features'] = [appartments_features]


In [0]:
# всякие штуки типо тип квартиры, площадь, какой санузел, тип ремонта
additional_features = []
for i in card_page.find_all(class_="a10a3f92e9--value--3Ftu5"):
  additional_features.append(i.get_text())

df['additional_features'] = [additional_features]

In [0]:
# цена аренды
rent_price = card_page.find(class_="a10a3f92e9--price_value--1iPpd").get_text().replace(u'\xa0', ' ')

In [0]:
df['rent_price'] = [rent_price]

In [0]:
rent_price_info = card_page.find(class_="a10a3f92e9--more_price_rent---5hwY").get_text().replace(u'\xa0', ' ')

In [0]:
df['rent_price_info'] = rent_price_info

In [0]:
additional_payment_info = card_page.find(class_="a10a3f92e9--description--2xRVn").get_text().replace(u'\xa0', ' ').split(",")

In [0]:
df['additional_payment_info'] = [additional_payment_info]

In [0]:
# телефоны
phones = card_page.find(class_="a10a3f92e9--container--35iPF").get_text().replace('Показать телефон', '').replace('Написать сообщение', '').split('+')

In [0]:
df['phones'] = [phones]

In [0]:
# название агентства
agency_name = card_page.find('h2', class_="a10a3f92e9--title--2gUWg").get_text()

In [0]:
df['agency_name'] = [agency_name]

In [0]:
import datetime

In [0]:
upload_d_t = card_page.find(class_="a10a3f92e9--container--3nJ0d").get_text()
view_statistics = card_page.find(class_="a10a3f92e9--link--1t8n1 a10a3f92e9--link--2mJJk").get_text()
if "сегодня" in upload_d_t:
  date = str(datetime.datetime.today())
elif 'вчера' in upload_d_t:
  date = str(datetime.datetime.today())
upload_d_t = [date, upload_d_t]
df['upload_d_t'] = [upload_d_t]

In [196]:
df

,title,adress,metro,apartments_area,living_apartmetn_area,kitche_area,apartments_level,year_of_construction,apartsments_text_subscription,appartments_features,additional_features,rent_price,rent_price_info,additional_payment_info,phones,agency_name,upload_d_t
0,"1-комн. апартаменты, 40 м²","Москва, ЦАО, р-н Басманный, Нижняя Красносельс...","[[[<path d=""M11.154 4L8 9.53 4.845 4 1.1 13.46...",40 м²,25 м²,5 м²,7 из 10,2015,"Покажу оперативно! ЖК ""Трибека"", корпус Fulton...","[Холодильник, Посудомоечная машина, Стиральная...","[Апартаменты, 25 м², 1 совмещенный, Дизайнерский]",70 000 ₽/мес.,Комм. платежи включены (без счётчиков),"[Залог 70 000 ₽, комиссия 50%, предоплата за...","[, 7 999 922-64-29, 7 985 998-50-24]",ID 320568,"[2020-01-11 23:40:23.500392, сегодня, 07:51]"


In [197]:
df['metro']

0    [[[<path d="M11.154 4L8 9.53 4.845 4 1.1 13.46...
Name: metro, dtype: object